In [1]:
import numpy as np
import pandas as pd
import string
import re
import os

# Plotting:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

# Import the seaborn palette, change for different applications.
from palettable.tableau import PurpleGray_6 as palette

sns.set_palette(palette.mpl_colors) #Set the seaborn palette
palette = palette.mpl_colors #Store the seaborn palette

# Plot output:
from matplotlib.backends.backend_pdf import PdfPages
import os

def readplatemap(platemap: str, levelnames: list = ['column', 'row', 'Well']) -> pd.DataFrame:
    ''' Reads in an Excel platemap to a dataframe.
    Parameters:
        platemap: path to .xlsx file with the platemap
        levelnames: list of the different levels in the platemap (seperated by commas)
    Return:
        DataFrame: dataframe of the annotated levelnames and the wells they belong to.
    '''
    log = "platemap read from: " + platemap
    platemap = pd.read_excel(platemap, sheetname = 0, header = 1, index_col = 1) #If the platemap is a .xlsx sheet read it in using read_excel:
    platemap = platemap.loc[platemap.index.isin(list(string.ascii_letters)), platemap.columns.isin(range(0,25))] #Remove any columns or rows that aren't named by integer or letter respectively:
    platemap.index.name = 'row' #Name the platemap index
    platemap = pd.melt(platemap.reset_index(col_fill = ['row']), id_vars='row', var_name='column') #Reshape the platemap into a long form
    platemap['Well'] = platemap['row'] + platemap['column'].astype(str) #concatonate to get the row column
    annotation = platemap['value'].str.split(pat = ',', expand = True) #split the annotations on commas to produce different levels
    annotation.columns = levelnames[:-1] #ignore the well column
    platemap = pd.concat([platemap, annotation], axis = 1).drop('value', axis = 1) #join the annotations with the wells again and remove the old value variable
    platemap = platemap.replace('NA', np.nan)
    platemap['Well'] = platemap['Well'].map(lambda x: x if len(x) == 3 else x[0]+'0'+x[1])
    return(platemap)

def parse_plate(xlsx: str, platemap:str, sheetname:str ='Raw', levelnames:list=['row','column','Well']) -> pd.DataFrame:
    ''' Reads in the plate data from an Excel sheet, annotates it with data from a platemap and transfers timestamps to hours'''
    # read in file:
    plate = pd.read_excel(xlsx, sheetname=sheetname)
    log = "Data read from " + xlsx + "."
    print(log)
    platemap = readplatemap('platemap.xlsx', levelnames=levelnames)
    plate = platemap.merge(plate)
    plate['measurement_step'] = plate['Tick'] % 12
    
    hours = plate['TimeStamp'].str.split(':', expand=True)
    hours.columns = ['hours', 'minutes', 'seconds']
    hours['year'] = '2018'
    hours['day'] = '1'
    hours['month'] = '1'
    hours = pd.to_datetime(hours)
    hours = hours-hours[0]
    hours = hours.dt.total_seconds()/60/60
    plate['Time (hrs)'] = hours
    return(plate)


def plot_variable(data, condition='strain', value = 'pH', correct=True):
    pdf = PdfPages('test.pdf')
    if correct:
        data = data.query('measurement_step == 0')
    # plot with hues
    sns.tsplot(data = data, condition = condition, unit = 'Well', time = 'Time (hrs)', value = value)
    plt.suptitle(condition)
    plt.show()

    # plot seperately
    for name,sub in data.groupby(by = ['strain']):
        fig, ax = plt.subplots()
        ax = sns.tsplot(ax = ax, data = sub, condition = condition, unit = 'Well', time = 'Time (hrs)', value = value)
        # ax2 =ax.twinx()
        # sns.tsplot(data = sub, condition = condition, unit = 'Well', time = 'Time (hrs)', value = 'pH')
        plt.suptitle(name)
        pdf.savefig()
        plt.show()
    pdf.close()
    return()

# how to get derivatives
# plate['pH_diff'] = plate.groupby('Well')['pH'].diff().diff()

In [5]:
# attempt at user input
xlsx = input(prompt='Path to data .xlsx:')
platemap = input(prompt='Path to platemap .xlsx:')


xlsx = '2018 03 29 replicate 02 09 assay KS.xlsx'
platemap = 'platemap.xlsx'
levelnames = ['strain','Well']

plate = parse_plate(xlsx = '2018 03 29 replicate 02 09 assay KS.xlsx',platemap = 'platemap.xlsx', levelnames = ['strain','Well'])

Data read from 2018 03 29 replicate 02 09 assay KS.xlsx.


In [2]:
value = input('Column to plot:')
condition = input(prompt='Column to group replicates by:')


value = 'pH'
condition = 'strain'
plot_variable(data=plate, condition=condition,value=value)

Column to plot:
Column to group replicates by:


NameError: name 'plate' is not defined

plate.query('Well == "A01"')['O2 (mmHg)'][0:36].plot()
plt.axvline(12)